In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
%cd /gdrive/My\ Drive/Secured/QICC/fakeNewsDataset

/gdrive/My Drive/Secured/QICC/fakeNewsDataset


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import itertools
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import os

In [0]:
df = pd.DataFrame(columns=['Headline','Description','Domain','Fake/Legit'])
df_test=pd.DataFrame(columns=['Headline','Description','Domain','Fake/Legit'])

In [0]:
df_eval=pd.DataFrame(columns=['Headline','Description','Domain','Fake/Legit'])

In [0]:
import glob
import re
files_train_fake=glob.glob('fake-training/*.txt')
files_train_legit=glob.glob('legit-training/*.txt')

In [0]:
file_eval=glob.glob('fake-news-real-news-eval/*.txt')

In [0]:
#import fake_training_dataset
for file_name in files_train_fake:
  with open(file_name, "r") as f:
    names_list = [l for l in (line.strip() for line in f) if l]
    if len(names_list)>1:
      df=df.append({'Headline':names_list[0],'Description': "".join(names_list[0:]), 'Domain':re.search(r"[a-zA-Z]*",file_name[14:]).group(), 'Fake/Legit':1}, ignore_index=True)
    else:
      df=df.append({ 'Description': names_list[0], 'Domain':re.search(r"[a-zA-Z]*",file_name[14:]).group(), 'Fake/Legit':1},ignore_index=True)

In [0]:
import codecs

In [0]:
for file_name in file_eval:
  with codecs.open(file_name, 'r', encoding='utf-8', errors='ignore')  as f:
    names_list = [l for l in (line.strip() for line in f) if l]
    if len(names_list)>1:
      df_eval=df_eval.append({'Headline':names_list[0],'Description': "".join(names_list[0:])}, ignore_index=True)
    else:
      df_eval=df_eval.append({ 'Description': names_list[0]}, ignore_index=True)

In [0]:
df.head()

,Unnamed: 0,Headline,Description,Domain,Fake/Legit
0,0,NaN,"""Tesco will not pay out any money to settle in...",biz,1
1,1,EU Applauds Deutsche Boerse's $14 Billion Take...,EU Applauds Deutsche Boerse's $14 Billion Take...,biz,1
2,2,Amazon to sell Middle East online retailer Souq,Amazon to sell Middle East online retailer Sou...,biz,1
3,3,Toshiba's Westinghouse creating thriving job m...,Toshiba's Westinghouse creating thriving job m...,biz,1
4,4,NaN,Ford has been forced by Donald Trump to pull o...,biz,1


In [0]:
#import legit_training_dataset
for file_name in files_train_legit:
  with open(file_name, "r") as f:
    names_list = [l for l in (line.strip() for line in f) if l]
    if len(names_list)>1:
      df=df.append({'Headline':names_list[0],'Description': "".join(names_list[0:]), 'Domain':re.search(r"[a-zA-Z]*",file_name[15:]).group(), 'Fake/Legit':0}, ignore_index=True)
    else:
      df=df.append({ 'Description': names_list[0], 'Domain':re.search(r"[a-zA-Z]*",file_name[15:]).group(), 'Fake/Legit':0},ignore_index=True)

In [0]:
files_test_fake=glob.glob('fake-test/*.txt')
files_test_legit=glob.glob('legit-test/*.txt')

In [0]:
#import fake_test_dataset
for file_name in files_test_fake:
  with open(file_name, "r") as f:
    names_list = [l for l in (line.strip() for line in f) if l]
    if len(names_list)>1:
      df_test=df_test.append({'Headline':names_list[0],'Description': "".join(names_list[0:]), 'Domain':re.search(r"[a-zA-Z]*",file_name[10:]).group(), 'Fake/Legit':1}, ignore_index=True)
    else:
      df_test=df_test.append({ 'Description': names_list[0], 'Domain':re.search(r"[a-zA-Z]*",file_name[10:]).group(), 'Fake/Legit':1},ignore_index=True)


#import legit_test_dataset
for file_name in files_test_legit:
  with open(file_name, "r") as f:
    names_list = [l for l in (line.strip() for line in f) if l]
    if len(names_list)>1:
      df_test=df_test.append({'Headline':names_list[0],'Description': "".join(names_list[0:]), 'Domain':re.search(r"[a-zA-Z]*",file_name[11:]).group(), 'Fake/Legit':0}, ignore_index=True)
    else:
      df_test=df_test.append({ 'Description': names_list[0], 'Domain':re.search(r"[a-zA-Z]*",file_name[11:]).group(), 'Fake/Legit':0},ignore_index=True)

In [0]:
X_train,X_test,y_train,y_test = df['Description'], df_test['Description'], df['Fake/Legit'], df_test['Fake/Legit']

In [0]:
#Applying tfidf to the data set
tfidf_vect = TfidfVectorizer(stop_words = 'english')
tfidf_train = tfidf_vect.fit_transform(X_train)
tfidf_test = tfidf_vect.transform(X_test)
tfidf_df = pd.DataFrame(tfidf_train.A, columns=tfidf_vect.get_feature_names())

In [0]:
clf = MultinomialNB() #Applying Naive Bayes

In [0]:
clf.fit(tfidf_train, y_train.astype('int')) # Fit Naive Bayes classifier according to X, y

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:

pred = clf.predict(tfidf_test)                     # Perform classification on an array of test vectors X.
score = metrics.accuracy_score(y_test.astype('int'), pred)
print("accuracy:   %0.3f" % score)
cm = metrics.confusion_matrix(y_test.astype('int'), pred)
print(cm)

accuracy:   0.750
[[19  5]
 [ 7 17]]


In [0]:
from sklearn.metrics import classification_report
print('REPORT:')
print(classification_report(y_test.astype(int), pred))

REPORT:
              precision    recall  f1-score   support

           0       0.73      0.79      0.76        24
           1       0.77      0.71      0.74        24

    accuracy                           0.75        48
   macro avg       0.75      0.75      0.75        48
weighted avg       0.75      0.75      0.75        48



In [0]:
tfidf_eval = tfidf_vect.transform(df_eval['Description'])

In [0]:
pred = clf.predict(tfidf_eval)                     # Perform classification on an array of test vectors X.


In [0]:
pred

array([0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1])

In [0]:
df_eval['Fake/Legit']=pd.Series(pred, index=df_eval.index)

Domain Prediction

In [0]:
df=pd.read_csv("training.csv")
df_test=pd.read_csv("test.csv")

In [0]:
X_train,X_test,y_train,y_test = df['Description'], df_test['Description'], df['Domain'], df_test['Domain']

In [0]:
#Applying tfidf to the data set
tfidf_vect = TfidfVectorizer(stop_words = 'english')
tfidf_train = tfidf_vect.fit_transform(X_train)
tfidf_test = tfidf_vect.transform(X_test)
tfidf_df = pd.DataFrame(tfidf_train.A, columns=tfidf_vect.get_feature_names())

In [0]:
clf = MultinomialNB() #Applying Naive Bayes
clf.fit(tfidf_train, y_train) # Fit Naive Bayes classifier according to X, y

pred = clf.predict(tfidf_test)                     # Perform classification on an array of test vectors X.
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
cm = metrics.confusion_matrix(y_test, pred)
print(cm)

accuracy:   0.771
[[4 0 0 0 0 4]
 [0 8 0 0 0 0]
 [4 0 3 0 1 0]
 [0 0 0 8 0 0]
 [0 0 1 0 7 0]
 [0 0 0 0 1 7]]


In [0]:
from sklearn.metrics import classification_report
print('REPORT:')
print(classification_report(y_test, pred))

REPORT:
              precision    recall  f1-score   support

         biz       0.50      0.50      0.50         8
         edu       1.00      1.00      1.00         8
       entmt       0.75      0.38      0.50         8
       polit       1.00      1.00      1.00         8
      sports       0.78      0.88      0.82         8
        tech       0.64      0.88      0.74         8

    accuracy                           0.77        48
   macro avg       0.78      0.77      0.76        48
weighted avg       0.78      0.77      0.76        48



In [0]:
pred = clf.predict(tfidf_eval)

In [0]:
pred

array(['edu', 'edu', 'edu', 'polit', 'entmt', 'polit', 'polit', 'entmt',
       'sports', 'polit', 'sports', 'sports', 'entmt', 'entmt', 'entmt',
       'entmt', 'polit', 'polit', 'polit', 'polit', 'tech', 'tech',
       'tech', 'tech', 'sports', 'tech', 'tech', 'entmt', 'sports', 'edu',
       'sports', 'sports', 'polit', 'biz', 'polit', 'biz', 'polit', 'biz',
       'polit', 'biz', 'polit', 'polit', 'polit', 'edu', 'edu', 'polit',
       'entmt', 'entmt', 'tech'], dtype='<U6')

In [0]:
df_eval['Domain']=pd.Series(pred, index=df_eval.index)

In [0]:
df_eval.head()

,File Name,Domain,Fake/Legit
0,asd01.blind.txt,edu,0
1,asd02.blind.txt,edu,1
2,asd03.blind.txt,edu,1
3,asd04.blind.txt,polit,1
4,bsb01.blind.txt,entmt,1


In [0]:
filesl=[]
for i in file_eval:
  filesl.append(i[25:]);


In [0]:
df_eval['File Name']=pd.Series(filesl, index=df_eval.index)

In [0]:
df_eval

,Headline,Description,Domain,Fake/Legit,File Name
0,Male teachers preparing to cover down on 'Day ...,Male teachers preparing to cover down on 'Day ...,edu,0,asd01.blind.txt
1,Colombia University Busts a Steve Harvey..,Colombia University Busts a Steve Harvey..When...,edu,1,asd02.blind.txt
2,Betsy DeVos's Education Hearing Brings Creatio...,Betsy DeVos's Education Hearing Brings Creatio...,edu,1,asd03.blind.txt
3,Trump's Uses His Big Brain,"Trump's Uses His Big BrainThe Senate, with bip...",polit,1,asd04.blind.txt
4,NaN,"ARCADIA, CAAfter the death of 3-year-old colt ...",entmt,1,bsb01.blind.txt
5,NaN,WASHINGTONExplaining that they faced a serious...,polit,0,bsb02.blind.txt
6,US President Donald Trump called Australian PM...,US President Donald Trump called Australian PM...,polit,0,bsb03.blind.txt
7,An activist involved in anti-government protes...,An activist involved in anti-government protes...,entmt,0,bsb04.blind.txt
8,Signed autograph of Michael Jordan's 'Basketba...,Signed autograph of Michael Jordan's 'Basketba...,sports,0,fgh01.blind.txt
9,SEXIST RORY MCILROY CALLS VOTE FOR FEMALE MEMB...,SEXIST RORY MCILROY CALLS VOTE FOR FEMALE MEMB...,polit,1,fgh02.blind.txt


In [0]:
df_eval=df_eval.drop(columns=['Headline', 'Description'])

In [0]:
df_eval=df_eval[['File Name', 'Domain', 'Fake/Legit']]

In [0]:
df_eval

,File Name,Domain,Fake/Legit
0,asd01.blind.txt,edu,0
1,asd02.blind.txt,edu,1
2,asd03.blind.txt,edu,1
3,asd04.blind.txt,polit,1
4,bsb01.blind.txt,entmt,1
5,bsb02.blind.txt,polit,0
6,bsb03.blind.txt,polit,0
7,bsb04.blind.txt,entmt,0
8,fgh01.blind.txt,sports,0
9,fgh02.blind.txt,polit,1


In [0]:
df_eval.to_csv('01Binary_Task1A.csv')